# Neural Network to Classify Handwritten Digits from MNIST Data set

## The Architecture

https://github.com/peterantonarosjr/NeuralNetFromScratch/blob/main/NN_Architecture.pdf

## Implementation

In [1]:
#These are going to be the only 3 imports I am allowing myself to create this network
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

2022-07-06 23:48:42.208656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 23:48:42.208678: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:
trainData = pd.read_csv("/home/peterjr/RepoCollections/NeuralNetFromScratch/MNIST_Data/train.csv")
testData = pd.read_csv("/home/peterjr/RepoCollections/NeuralNetFromScratch/MNIST_Data/test.csv")

In [18]:
#We need to split training data into X and y
trainData.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#Taking a look at X_train
X_train = trainData.loc[:, trainData.columns != 'label']
X_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#Taking a look at y_train
y_train = trainData[['label']]
y_train.head()

,label
0,1
1,0
2,1
3,4
4,0


In [34]:
#Convert to numpy arrays
X_train = np.array(X_train).T
y_train = np.array(y_train)
display(X_train.shape)
display(y_train.shape)

(784, 42000)

(42000, 1)

In [35]:
#Testing data
X_test = np.array(testData).T
X_test.shape

(784, 28000)

In [ ]:
#Defining functions related to our architecture above

#Set the initial parameters for the network
def initParams():
    return

#Define a forward propagation function to move inputs -> network
def forwardProp():
    return

#Define a backward propagation function to move inputs <- network
def backwardProp():
    return 

#Define a function to update our parameters during main loop
def updateParams():
    return

#Define a function to generate predictions with CURRENT weights
def makePredictions():
    return

#Define a function to calculate error with CURRENT weights
def getError():
    return:

#Define a gradient descent as our 'main loop'
def gradDescent():
    initParams()
    makePredictions()
    getError()
    
    #Start loop here
    forwardProp()
    backwardProp()
    updateParams()
    makePredictions()
    getError()
    
    #Add error to a plot to visualize over iterations 'epochs'
    
    #End loop here
    
    #Returns weights and anything else we might want
    return

In [ ]:
#Call on gradDescent() and watch it go!